In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import keras 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping 
from keras.wrappers.scikit_learn import KerasRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
file_name = '/kaggle/input/penmanshiel-2016/Turbine_Data_Penmanshiel_01_2016-06-06_-_2017-01-01_1042.csv'
df_ori = pd.read_csv(file_name, skiprows=[i for i in range(9)], sep=",", header=0) #, engine='python')
df_ori['datetime']=pd.to_datetime(df_ori['# Date and time'])
df_ori = df_ori.drop('# Date and time', axis=1)
# df_ori = df_ori.set_index('datetime')
df = df_ori
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
col_dtypes = {}
for col_name in df.columns:
    cur_col_dtype = str(df[col_name].dtypes)
    if cur_col_dtype not in col_dtypes:
        col_dtypes[cur_col_dtype] = [col_name]
    else:
        col_dtypes[cur_col_dtype].append(col_name)
col_dtypes.keys()

In [ ]:
df_null_sum = df.isnull().sum() #[:100]
sp = sns.lineplot(x=[i for i in range(len(df_null_sum))], y=df_null_sum, data=df_null_sum)
sp.set_ylabel('Number of null values')
sp.set_xlabel('Column index')

In [ ]:
#Removing columns with high number of null values
max_number_of_nans = 5000
col_large_nulls = []
for col_name in df_ori.columns:
    if df[col_name].isnull().sum() > max_number_of_nans:
        col_large_nulls.append(col_name)
df = df.drop(df.columns[df.apply(lambda col: col.isnull().sum() > max_number_of_nans)], axis=1)
len(col_large_nulls) # 103 columns have null values > 5000 while 193 columns for the other



In [ ]:
df.shape

In [ ]:
sorted_null_values = df.apply(lambda col: col.isnull().sum()).sort_values()

In [ ]:
sorted_null_values[:35]

In [ ]:
sns.lineplot(x=[i for i in range(len(sorted_null_values.index))], y=len(df)-sorted_null_values)

In [ ]:
df['Wind speed (m/s)']

In [ ]:
df['Wind speed (m/s)'].isnull()

In [ ]:
df[df['Wind speed (m/s)'].isnull()].index

In [ ]:
sns.lineplot(data=df[df.columns[:10]])

In [ ]:
cname =  'Time-based Contractual Avail.'
# 'Production-based Contractual Avail.' # (28824.4546776649, array([1., 0., nan, 0.11822272, 0.18001452, 0.02624681, 0.87575854, 0.31745465, 0.18256907, 0.35850908, 0.39590227]))
#'Long Term Wind (m/s)'# (207361.4, array([5.8, 5.6, 6. , 6.9, 7.5, 8.1, 8.3]))
#'Energy Import (kWh)'# (2187.0, array([ 0.,  1., nan,  3.,  2., 20.]))
# 'Data Availability'#1; 
# 'Lost Production to Curtailment (Birds) (kWh)'#0' 
# Available Capacity for Production (Planned) (kW)'#2050; 
# 'Lost Production to Curtailment (Sector Management) (kWh)'#0; 
# 'Lost Production to Curtailment (Technical) (kWh)'#0;'
# Lost Production to Curtailment (Marketing) (kWh)'#0;
# 'Lost Production to Curtailment (Boat Action) (kWh)'#0; 
# 'Lost Production to Curtailment (Grid Constraint) (kWh)'#0;
# 'Lost Production to Curtailment (Bats) (kWh)'#0; 
# 'Lost Production to Curtailment (Shadow) (kWh)'#0; 
# 'Compensated Lost Production (kWh)'#0;
df_ori[cname].sum(), df_ori[cname].unique()

In [ ]:
#Deleting more columns
#1. If a column is about 'Availability ...' then it will not contribute much
#2. If a column only few unique value(s)
df = df.drop(df.columns[df.apply(lambda col: len(col.unique())  <= 1000)], axis=1)
unique_values = df.apply(lambda col: len(col.unique()))

sp = sns.barplot(y=list(unique_values.index), x=unique_values.tolist())
for container in sp.containers:
    sp.bar_label(container)
sp.set_xlabel('Number of unique values')

In [ ]:
df.apply(lambda col: len(col.unique()))#.sort_values(ascending=True)

In [ ]:
sns.lineplot(data=df[['Power (kW)', 'Energy Theoretical (kWh)']])

In [ ]:
df.shape, df.isnull().any().sum(), df.isna().any().sum() # ((29987, 86), 78, 78)
# 86-78=8 columns do not null values

In [ ]:
#columns with no null values
df.isnull().any().sort_values()[:9]

In [ ]:
#fill NaNs with column medians in each column 
df = df.fillna(df.median())
#Check if all null or NaN have been replaced
df.isnull().any().sum()
#df.isnull().any()

In [ ]:
sns.lineplot(data=df[['Power (kW)', 'Energy Theoretical (kWh)']])

In [ ]:
df.apply(lambda col: len(col.unique()))

In [ ]:
df['Power (kW)'].median()

In [ ]:
# Put target 'Power (kW)' in the first column
target_col = 'Power (kW)'
new_cols = [i for i in df.columns]
new_cols.remove(target_col)
new_cols = [target_col] + new_cols

df = df[new_cols]
df.head()

In [ ]:
# for col_name in df.columns:
#     print("{} \t {}".format(df[col_name].isnull().sum(),col_name))

In [ ]:
df[['Power (kW)','Energy Theoretical (kWh)']]

In [ ]:
# usd = df['Power (kW)'] # df[df['Currency']=='US dollar'].reset_index(drop=True)
# sns.lineplot(x=[i for i in range(len(df))], y='Power (kW)', data=df)

In [ ]:
sns.lineplot(x=[i for i in range(len(df))], y='Wind speed (m/s)', data=df)

In [ ]:
#from datetime import datetime
df['week_day']=df['datetime'].dt.dayofweek
df['date']=df['datetime'].dt.day
df['month']=df['datetime'].dt.month
df['hour']=df['datetime'].dt.hour
df.head()

In [ ]:
# pip install autoviz

In [ ]:
# from autoviz.AutoViz_Class import AutoViz_Class
# AV = AutoViz_Class()
# d = AV.AutoViz(df)
# d = AV.AutoViz('', ',', 'Power (kW)', df)

In [ ]:
if 'datetime' in df.columns:
    df = df.set_index('datetime')

#split data into training and testing
# split_point = '2016-06-16 02:10:00' # '2016-12-01'
split_point = int(len(df)*0.80)

df = df.iloc[:,0:5]
# train, test = df.loc[:split_point], df.loc[split_point:]
train, test = df[:split_point], df[split_point:]

In [ ]:
df.shape

In [ ]:
train.tail(1)

In [ ]:
test.head(5)

In [ ]:
print(len(train))
print(len(test))

In [ ]:
# scale the data using MinMax Scaler from -1 to 1 as LSTM has a default tanh activation function
SCALER = MinMaxScaler(feature_range=(-1,1))

scaler = SCALER.fit(train.to_numpy())

train_scaled = scaler.transform(train.to_numpy())
test_scaled = scaler.transform(test.to_numpy())

In [ ]:
# create a function to split the datasets into two week windows
# timestep = 24*7*2 # 24hours,7days,2weeks
timestep = 24*7*1 # 24hours,7days,1weeks

def create_dataset(dataset, timestep=timestep):
    """
    Function which creates two week chunks of x_train data, and a single
    value for y_train.
    """
    X, y = [], []
    for i in range(len(dataset)):
        target_value = i + timestep
        if target_value == len(dataset):
            break
        feature_chunk, target = dataset[i:target_value, 1:], dataset[target_value, 0]
        X.append(feature_chunk)
        y.append(target)
    
    return np.array(X), np.array(y) 

In [ ]:
#create x_train, y_train, X_test,y_test
X_train, y_train = create_dataset(train_scaled)
X_test, y_test = create_dataset(test_scaled)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# use sample of th data to train network to have a rough understanding of hyperparameters
samp_len = int(len(X_train)*0.5)

X_sample_train, y_sample_train = X_train[:samp_len], y_train[:samp_len]

In [ ]:
print(X_sample_train.shape)
print(y_sample_train.shape)

In [ ]:
# create X_train, y_train, X_test, y_test datasets
# create a function to build a stacked LSTM model
# input needs to be [samples, timesteps, features]
def create_model(X_train, y_train):
    units = 32
    dropout = 0.05 # https://github.com/keras-team/keras/blob/v2.13.1/keras/layers/rnn/lstm.py#L382
    epochs = 3 #35
    batch_size = 14
    optimizer = keras.optimizers.Adam(learning_rate=0.0005)
    early_stopping = EarlyStopping(patience=7, monitor='loss')

    model = keras.Sequential()

    model.add(LSTM(units=units, dropout=dropout, return_sequences=True,
                   input_shape=(X_train.shape[1], X_train.shape[2])))

    model.add(LSTM(units=units, dropout=dropout))

    model.add(Dense(units=1))

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    history = model.fit(X_train, y_train, validation_split=0.3, shuffle=False,
              epochs=epochs, batch_size=batch_size, verbose=1, callbacks=[early_stopping])

    return model, history

In [ ]:
# function to predict a single value 
def single_prediction(model, history, timestep=timestep):
        
        history = np.array(history)
        history = history.reshape(history.shape[0]*history.shape[1], history.shape[2])
        
        input_value = history[-timestep:]
        input_value = input_value.reshape(1, input_value.shape[0], input_value.shape[1])
        
        yhat = model.predict(input_value, verbose=0)
        return yhat

In [ ]:
# function which takes first test chunk, makes a prediction, add the test chunk back into training data 
#to make next prediction

def walk_forward_prediction(X_train, y_train, X_test, timestep):
    
    MODEL, history = create_model(X_train=X_train, y_train=y_train)
    hist_train = [i for i in X_train]
    predictions = []
    
    for i in range(len(X_test)):
        test = X_test[i]
        yhat = single_prediction(model=MODEL, history=hist_train, timestep=timestep)
        predictions.append(yhat) 
        hist_train.append(test)
    
    return predictions, history, MODEL

In [ ]:
def prior_inverse(features, targets):
    '''
    Append prediction value to test dataset and return a test shape format.
    '''
    dataset = []
    
    for i in range(features.shape[0]):
        last_row, target = features[i][0], targets[i]
        appended = np.append(last_row, target)
        dataset.append(appended)
    
    return np.array(dataset) 

In [ ]:
#run experiemnt returning the real, predicted values
def experiment(X_train, y_train, X_test, timestep):
    
    pred_seq, history, MODEL = walk_forward_prediction(X_train, y_train, X_test, timestep)
    
    pred_seq = np.array(pred_seq).reshape(-1)

    pred = prior_inverse(X_test, pred_seq)
    real = prior_inverse(X_test, y_test)

    inv_pred = scaler.inverse_transform(pred)
    inv_real = scaler.inverse_transform(real)

    power_pred = inv_pred[:,-1]
    power_real = inv_real[:,-1]
    
    return power_real, power_pred, history, MODEL

In [ ]:
power_real, power_pred, history, MODEL = experiment(X_train, y_train, X_test, timestep)

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
#plot validation and training convergence graph
plt.figure(figsize=(10,5))
plt.plot(loss, label='train')
plt.plot(val_loss, label='validation')
plt.legend()
plt.xlabel('epochs')
plt.ylabel('MSE')
plt.title('LSTM Training Validation Loss')
plt.tight_layout()
plt.show()

In [ ]:
x_plot = test[timestep:].index
pred_df = pd.DataFrame({'Date':x_plot, 'Prediction': power_pred, 'True': power_real})
pred_df.set_index('Date', inplace=True)

In [ ]:
pred_df2 = pred_df['2016-12-16 02:10:00':'2016-12-22 02:10:00']

In [ ]:
#plot predictions
# pred_df2.plot(rot='60',figsize=(10,5)) # ValueError: rotation must be 'vertical', 'horizontal' or a number, not 60
# pred_df2.plot(rot='horizontal',figsize=(10,5)) # x=date, y=power
pred_df2.plot(rot=60,figsize=(10,5)) # x=date, y=power
plt.title('Predicted Power vs Actual Power with LSTM an Model.')
plt.ylabel('Power(KWh)')
plt.tight_layout()
plt.show()

In [ ]:
#compute metrics
rmse = np.sqrt(mean_squared_error(pred_df2['True'], pred_df2['Prediction']))
mae = mean_absolute_error(pred_df2['True'], pred_df2['Prediction'])
r2 = r2_score(pred_df2['True'], pred_df2['Prediction'])
print('RMSE: {}\nMAE: {}\nR2: {}'.format(round(rmse,2),round(mae,2), round(r2,2)))

In [ ]:
#FB Prophet Model
# !pip install prophet

In [ ]:
from prophet import Prophet
# df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')
df_fb = df_ori
df_fb.head()

In [ ]:
df_fb = df_ori[['datetime', 'Power (kW)']]
# df_fb = df_fb.set_index('datetime')
df_fb = df_fb.rename(columns={"datetime": "ds", "Power (kW)": "y"})
df_fb.head()

In [ ]:
m = Prophet()
m.fit(df_fb)

In [ ]:
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly
plot_plotly(m, forecast)

In [ ]:
print(min(df_fb['ds']))
print(max(df_fb['ds']))